# Eval Starter Kit Notebook

In [ ]:
import os
import sys

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

print(f"This is the repo dir {repo_dir}")

In [ ]:
import pandas as pd
from utils.eval.rag_eval import (
    RAGEvaluator,
    RAGEvalConfig,
    load_pipeline,
    load_eval_dataframe,
    evaluate,
)
from langchain_community.llms.sambanova import SambaStudio
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

In [ ]:
# Use Case 1: CSV file with pre-generated answers, no context
# Load config
config_path = "../../utils/eval/config.yaml"
config = RAGEvalConfig(config_yaml_path=config_path)

# Create evaluator
eval_llms = []
for conf in config.eval_llm_configs:
    print("conf:", conf)
    llm_name, llm_config = config.get_llm_config(conf)
    print("llm_name:", llm_name)
    print("llm_config:", llm_config)
    eval_llm = SambaStudio(**llm_config)
    eval_llms.append((llm_name, eval_llm))

print("eval_llms:", eval_llms)

eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, config_path)

In [ ]:
eval_df = pd.read_csv("../data/tepsys_eng_tran.csv")
results1 = evaluator.evaluate(eval_df)
print("Results 1:", results1)

In [ ]:
# Use Case 2: CSV file with pre-generated answers and context, add context_col to eval_dataset yaml section
# Re- Load config after changes
eval_df = pd.read_csv("../data/res.csv")
config_path = "../../utils/eval/config.yaml"
config = RAGEvalConfig(config_yaml_path=config_path)

# Create evaluator
eval_llms = []
for conf in config.eval_llm_configs:
    print("conf:", conf)
    llm_name, llm_config = config.get_llm_config(conf)
    print("llm_name:", llm_name)
    print("llm_config:", llm_config)
    eval_llm = SambaStudio(**llm_config)
    eval_llms.append((llm_name, eval_llm))

print("eval_llms:", eval_llms)

eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, config_path)


results2 = evaluator.evaluate(eval_df)
print("Results 2:", results2)

In [ ]:
# Use Case 3: CSV file without answers, generate with pipelines, no context
# Re- Load config after changes
eval_df = pd.read_csv("../data/tepsys_eng_tran.csv")
config_path = "../../utils/eval/config.yaml"
config = RAGEvalConfig(config_yaml_path=config_path)

# Create evaluator
eval_llms = []
for conf in config.eval_llm_configs:
    print("conf:", conf)
    llm_name, llm_config = config.get_llm_config(conf)
    print("llm_name:", llm_name)
    print("llm_config:", llm_config)
    eval_llm = SambaStudio(**llm_config)
    eval_llms.append((llm_name, eval_llm))

print("eval_llms:", eval_llms)

eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, config_path)

pipelines = [
    load_pipeline((llm_name, SambaStudio(**llm_config)), config)
    for llm_name, llm_config in [
        config.get_llm_config(conf) for conf in config.llm_configs
    ]
]
results3 = evaluator.evaluate(eval_df, pipelines)
print("Results 3:", results3)

In [ ]:
# Use Case 4: CSV file without answers, generate with pipelines, with context from vector DB
# Re- Load config after changes
eval_df = pd.read_csv("../data/res.csv")
config_path = "../../utils/eval/config.yaml"
config = RAGEvalConfig(config_yaml_path=config_path)

# Create evaluator
eval_llms = []
for conf in config.eval_llm_configs:
    llm_name, llm_config = config.get_llm_config(conf)
    eval_llm = SambaStudio(**llm_config)
    eval_llms.append((llm_name, eval_llm))


eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, config_path)

eval_df = pd.read_csv("../data/res.csv")
pipelines = [
    load_pipeline((llm_name, SambaStudio(**llm_config)), config)
    for llm_name, llm_config in [
        config.get_llm_config(conf) for conf in config.llm_configs
    ]
]
results4 = evaluator.evaluate(eval_df, pipelines)
print("Results 4:", results4)

In [ ]:
# Running the evaluation using CLI
config_path = "path/to/config.yaml"
eval_csv_path = "../data/res.csv"
config_path = "../../utils/eval/config.yaml"

# Command without generation
command_no_gen = [
    "python",
    "evaluate.py",
    "--config",
    config_path,
    "--eval_csv",
    eval_csv_path,
]

# Command with generation
command_with_gen = [
    "python",
    "evaluate.py",
    "--config",
    config_path,
    "--eval_csv",
    eval_csv_path,
    "--generation",
]

# Execute command without generation
result_no_gen_cli = subprocess.run(command_no_gen, capture_output=True, text=True)
print("CLI results without generation:")
print(result_no_gen_cli.stdout)

# Execute command with generation
result_with_gen_cli = subprocess.run(command_with_gen, capture_output=True, text=True)
print("CLI results with generation:")
print(result_with_gen_cli.stdout)